In [186]:
import pandas as pd
import numpy as np
import pymysql

In [187]:
conexion = pymysql.connect(
    host="localhost",
    user="root",
    # password="123456",
    database="cidia_pp2",
    port=3306)
query = 'SELECT * FROM tiempos_por_tecla where (tecla <= 47 or tecla >= 58)  and tecla <> 44 and tecla <> 46 and modo = 1 order by tiempo_id'    
query1 = 'SELECT usuario,usuario_id FROM usuarios ORDER BY usuario_id'
query2 = 'select usuarios.usuario,usuarios.usuario_id,(select count(distinct practica) from tiempos_por_tecla where tiempos_por_tecla.usuario=usuarios.usuario_id and modo=1) as frases_largas,(select count(distinct practica) from tiempos_por_tecla where tiempos_por_tecla.usuario=usuarios.usuario_id and modo=2) as frases_cortas from usuarios order by (frases_largas+frases_cortas) desc'

df_usuarios = pd.read_sql(query1, conexion)
df_datos = pd.read_sql(query, conexion)
df_cargas =pd.read_sql(query2, conexion)

In [188]:
df_datos.head()

,tiempo_id,usuario,practica,tecla,up,down,anterior,anterior_down,modo
0,1,1,1,69,43,0,0,0,1
1,2,1,1,83,237,157,69,0,1
2,3,1,1,84,458,330,83,157,1
3,4,1,1,65,571,426,84,330,1
4,5,1,1,32,650,572,65,426,1


### Se calculan nuevas colmnas con restas entre UP, DOWN y DOWN ANTERIOR

In [189]:
df_datos['presion'] = df_datos.up - df_datos.down
df_datos.head()

,tiempo_id,usuario,practica,tecla,up,down,anterior,anterior_down,modo,presion
0,1,1,1,69,43,0,0,0,1,43
1,2,1,1,83,237,157,69,0,1,80
2,3,1,1,84,458,330,83,157,1,128
3,4,1,1,65,571,426,84,330,1,145
4,5,1,1,32,650,572,65,426,1,78


In [190]:
df_datos['down_entre_teclas'] = df_datos.down - df_datos.anterior_down
df_datos.head()

,tiempo_id,usuario,practica,tecla,up,down,anterior,anterior_down,modo,presion,down_entre_teclas
0,1,1,1,69,43,0,0,0,1,43,0
1,2,1,1,83,237,157,69,0,1,80,157
2,3,1,1,84,458,330,83,157,1,128,173
3,4,1,1,65,571,426,84,330,1,145,96
4,5,1,1,32,650,572,65,426,1,78,146


In [191]:
df_datos['up_down_entre_teclas'] = df_datos.up - df_datos.anterior_down
df_datos.head()

,tiempo_id,usuario,practica,tecla,up,down,anterior,anterior_down,modo,presion,down_entre_teclas,up_down_entre_teclas
0,1,1,1,69,43,0,0,0,1,43,0,43
1,2,1,1,83,237,157,69,0,1,80,157,237
2,3,1,1,84,458,330,83,157,1,128,173,301
3,4,1,1,65,571,426,84,330,1,145,96,241
4,5,1,1,32,650,572,65,426,1,78,146,224


## Analisis de los datos

### Se crea DF para ingresar a modelos

In [193]:
u = None
p = None
cont = -1
teclas = {}
temp = {}
for i in df_datos.usuario.unique():
    df_datos_usuario = df_datos[df_datos.usuario == i]

    for j in df_datos_usuario.practica.unique():
        cont += 1
        teclas = {}
        df_datos_practica = df_datos_usuario[df_datos_usuario.practica == j]

        if cont == 0:
            temp['usuario'] = [i]
            temp['practica'] = [j]
        else:
            temp['usuario'].append(i)
            temp['practica'].append(j)

        for k in df_datos_practica.index:
            if df_datos_practica.loc[k].tecla in teclas:
                teclas[df_datos_practica.loc[k].tecla] += 1
            else:
                teclas[df_datos_practica.loc[k].tecla] = 0
            
            nom_col = chr(df_datos_practica.loc[k].tecla) + str(teclas[df_datos_practica.loc[k].tecla])

            if cont == 0:
                # temp['kd_'+nom_col] = [df_datos_practica.loc[k].down]
                # temp['ku_'+nom_col] = [df_datos_practica.loc[k].up]
                temp['kp_'+nom_col] = [df_datos_practica.loc[k].presion]
                #temp['kdet_'+nom_col] = [df_datos_practica.loc[k].down_entre_teclas]
                temp['kudet_'+nom_col] = [df_datos_practica.loc[k].up_down_entre_teclas]

            else:
                # temp['kd_'+nom_col].append(df_datos_practica.loc[k].down)
                # temp['ku_'+nom_col].append(df_datos_practica.loc[k].up)
                temp['kp_'+nom_col].append(df_datos_practica.loc[k].presion)
                #temp['kdet_'+nom_col].append(df_datos_practica.loc[k].down_entre_teclas)
                temp['kudet_'+nom_col].append(df_datos_practica.loc[k].up_down_entre_teclas)
df1 = pd.DataFrame(temp)
df2 = pd.DataFrame(temp)

### Solo se utilizan las columnas calculadas ya que son restas de las columnas originales UP y DOWN

In [194]:
df1

,usuario,practica,kp_E0,kudet_E0,kp_S0,kudet_S0,kp_T0,kudet_T0,kp_A0,kudet_A0,...,kp_U11,kudet_U11,kp_A25,kudet_A25,kp_R22,kudet_R22,kp_I11,kudet_I11,kp_O13,kudet_O13
0,1,1,43,43,80,237,128,301,145,241,...,112,192,158,272,113,240,96,176,158,191
1,1,2,32,32,80,224,111,288,128,207,...,97,242,161,258,97,224,143,240,112,191
2,1,3,45,45,97,238,142,336,144,255,...,111,222,157,238,108,221,111,172,128,191
3,1,4,63,63,127,254,128,336,161,241,...,80,144,176,272,96,224,127,207,112,176
4,1,5,64,64,97,256,109,288,128,222,...,95,175,159,271,96,224,111,177,111,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,93,5,241,241,144,288,151,280,120,231,...,80,216,136,240,96,184,135,192,72,159
443,93,6,256,256,145,241,184,344,159,296,...,78,207,105,208,104,176,152,216,96,200
444,93,7,247,247,104,200,160,272,144,264,...,71,208,136,200,104,191,96,152,88,144
445,93,8,169,169,135,232,167,295,153,272,...,88,201,72,160,71,1975,128,176,135,215


### Se verifican valores negativos o ceros

In [195]:
(df1<0).sum().sum()

0

In [196]:
(df1==0).sum().sum()

3

In [538]:
#df_cargas

In [198]:
usuarios = df_datos['usuario'].unique()
usuarios

array([ 1, 64, 69, 66, 67, 72, 73, 68, 74, 75, 70, 65, 76, 62, 77, 79, 81,
       82, 83, 84, 85, 80, 89, 87, 91, 90, 93, 92], dtype=int64)

### Determinacion de parametros e hiperparametros

In [643]:
grillaC = [2.5e-7,5e-7,1e-6,2.5e-6,5e-6,1e-5,2.5e-5,5e-5,1e-4,2.5e-4,5e-4,1e-3,2.5e-3,5e-3,1e-2,2.5e-2,5e-2,1e-1,2.5e-1,5e-1,7.5e-1,1,1.5,2,5,10,25,50,100,500,1000]
Profundidad = 250
Vecino = 50
usuarios_seleccionados = [ 1, 64, 69, 66, 67, 72, 73, 68, 74, 75, 70, 65, 76, 62, 77, 79, 81, 82, 83, 84, 85, 80, 89, 87, 91, 90, 93, 92]
Cantidad_de_cargas_x_usuario = 12
Cargas_min_por_usuario = 12
Porcentaje_columnas = 100
test_size = 30

In [644]:
if usuarios_seleccionados != 0:
    df_seleccion = df1[df1['usuario'].isin(usuarios_seleccionados)]
else:
    df_seleccion = df1

### Se eliminan los usuarios con pocas cargas:

#### Usuarios totales:

In [645]:
usuarios = df_datos['usuario'].value_counts()
usuarios.count()

28

In [646]:
cargas_min = Cargas_min_por_usuario
df2 = df_seleccion.groupby(['usuario']).count()>=cargas_min
df3= df2[df2.kp_E0 >0]
df_carga_ok = df_seleccion[df_seleccion['usuario'].isin(df3.index)]

In [647]:
usuarios_carga = df_carga_ok['usuario'].value_counts()
usuarios_carga.count()

15

#### Eliminacion

In [648]:
cargas_min = Cargas_min_por_usuario
df2 = df_seleccion.groupby(['usuario']).count()>=cargas_min
df3= df2[df2.kp_E0 >0]
df_carga_ok = df_seleccion[df_seleccion['usuario'].isin(df3.index)]

In [649]:
df_carga_ok2 = pd.DataFrame(columns = df_carga_ok.columns)
for i in df_carga_ok['usuario'].unique():
    df_prov = df_carga_ok[df_carga_ok.usuario == i].iloc[0:Cantidad_de_cargas_x_usuario]
    df_carga_ok2 = pd.concat([df_carga_ok2, df_prov], ignore_index = True)
df_carga_ok = df_carga_ok2
    

#### Usuarios con cargas sufucientes para el ingreso a los modelos

In [650]:
usuarios = df_carga_ok['usuario'].value_counts()
usuarios.count()

15

#### Se eliminan columnas buscando el mejor resultado con menos carga y evitando el overfiting

In [651]:
hasta_col=int(df_carga_ok.shape[1]*Porcentaje_columnas/100)
df = df_carga_ok.iloc[:,0:hasta_col]

## Division en Train y Test

In [652]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import f1_score, confusion_matrix

In [653]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(axis=1,columns='usuario'),df['usuario'],test_size = test_size/100,random_state=123)
y_train = y_train.astype(str)

## Normalizacion de los datos

In [654]:
# Importamos las librerías para lalimpieza de datos
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

In [655]:
# Creamos los imputadores:
#cols_num = X_train.select_dtypes(np.number).columns
cols_num = X_train.columns
t_norm = ("normalizador",MinMaxScaler(feature_range=(0, 1)),cols_num)
#Se genera el Transformador de Columnas para aplicar las transformaciones
transformador_columnas = ColumnTransformer(transformers=[t_norm], remainder='passthrough')
# Entrenamos el Transformador de Columnas con fit sobre el X_train
transformador_columnas.fit(X_train)
# Lo aplicamos sobre el X_train y sobre el X_test
X_train_transformado = transformador_columnas.transform(X_train)
X_test_transformado = transformador_columnas.transform(X_test)
# Determinamos el orden de las columnas:
cols_orden = cols_num.tolist()

In [656]:
X_train_transformado.shape

(126, 445)

In [657]:
X_test_transformado.shape

(54, 445)

In [658]:
X_train_norm=pd.DataFrame(X_train_transformado, columns = cols_orden) 
X_test_norm=pd.DataFrame(X_test_transformado, columns = cols_orden)

In [659]:
X_train_norm.shape

(126, 445)

In [660]:
X_test_norm.shape

(54, 445)

#### Division en Validation Train y Validation Test

In [661]:
X_valid_train,X_valid_test,y_valid_train,y_valid_test=train_test_split(X_train_norm,y_train,test_size = test_size/100,random_state=123)

### Modelo Regresión Logística sin Regularización

In [662]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

In [663]:
#Creamos el modelo
modelo_valid_RLog = linear_model.LogisticRegression(max_iter= 22000,penalty='none',fit_intercept=True, random_state=123)
#Genero un DataFrame para incluir el resultado ya que este modelo al ser no paramétrico no necesita iteraciones
Arr_valid_RLog= pd.DataFrame(columns=['Hiperpar','Acc_valid_RLog', 'F1_valid_RLog'])
#Entrenamos el modelo de Regresión Logística
modelo_valid_RLog.fit(X_valid_train,y_valid_train)
#Obtenemos los coeficientes del modelo de Regresión Logística
y_valid_pred = modelo_valid_RLog.predict(X_valid_test)
Acc_valid_RLog=round(accuracy_score(y_valid_test, y_valid_pred),3)
F1_valid_RLog=round(f1_score(y_valid_test,y_valid_pred, average='macro'),3)
#Incluyo el resultado en el DataFrame creado al comienzo
Arr_valid_RLog = Arr_valid_RLog.append({'Hiperpar': 'xx','Acc_valid_RLog': Acc_valid_RLog, 'F1_valid_RLog': F1_valid_RLog}, ignore_index=True)

In [664]:
Arr_valid_RLog

,Hiperpar,Acc_valid_RLog,F1_valid_RLog
0,xx,0.974,0.981


### Modelo Regresión Logística con Regularización Ridge

In [665]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge

In [666]:
#Buscamos la mejor C de Ridge
#Genero un DataFrame para incluir la totalidad de los resultados de las iteraciones del for
Arr_Log_Ridge= pd.DataFrame(columns=['C','Acc_Log_Ridge', 'F1_Log_Ridge'])

for c in grillaC:
    Log_Ridge = LogisticRegression(penalty='l2', C=c, max_iter=10000, multi_class='ovr') 
#Entrenamos el modelo de Ridge (Clasificación)
    Log_Ridge.fit(X_valid_train,y_valid_train)            # entrenamos
#Obtenemos los coeficientes del modelo de Lasso (Clasificación)
    y_pred_Ridge = Log_Ridge.predict(X_valid_test)          #pronosticamos
    AC_Log_Ridge = Log_Ridge.score(X_valid_test,y_valid_test)  # evaluamos Accuracy
    F1_Log_Ridge = f1_score(y_valid_test,y_pred_Ridge, average="macro") # evaluamos F1
#Incluyo los resultados de las iteraciones en el DataFrame creado antes del for
    Arr_Log_Ridge = Arr_Log_Ridge.append({'C': c,'Acc_Log_Ridge': AC_Log_Ridge, 'F1_Log_Ridge': F1_Log_Ridge}, ignore_index=True)

In [667]:
#Arr_Log_Ridge

In [668]:
#Busco el "mejor resultado" en el DataFrame creado tras las iteraciones y asigno nuevo nombre para su uso en la comparacion 
#entre los mejores resultados de cada modelo
AC_L_Ridge = Arr_Log_Ridge.nlargest(1,'Acc_Log_Ridge')
ARRidge = AC_L_Ridge.set_index('C')
ARRidge

,Acc_Log_Ridge,F1_Log_Ridge
C,,
25.0,1.0,1.0


### Modelo Regresión Logística con Regularización Lasso

In [669]:
from sklearn.linear_model import Lasso

In [670]:
#Buscamos la mejor C de Lasso
#Genero un DataFrame para incluir la totalidad de los resultados de las iteraciones del for
Arr_Log_Lasso= pd.DataFrame(columns=['C','Acc_Log_Lasso', 'F1_Log_Lasso'])

for c in grillaC:
    Log_Lasso = LogisticRegression(penalty='l1', C=c, max_iter=10000, multi_class='ovr', solver='liblinear') 
#Entrenamos el modelo de Lasso (Clasificación)
    Log_Lasso.fit(X_valid_train,y_valid_train)            # entrenamos
#Obtenemos los coeficientes del modelo de Lasso (Clasificación)
    y_pred_Lasso = Log_Lasso.predict(X_valid_test)          #pronosticamos
    AC_Log_Lasso = Log_Lasso.score(X_valid_test,y_valid_test)  # evaluamos Accuracy
    F1_Log_Lasso = f1_score(y_valid_test,y_pred_Lasso, average="macro") # evaluamos F1
#Incluyo los resultados de las iteraciones en el DataFrame creado antes del for
    Arr_Log_Lasso = Arr_Log_Lasso.append({'C': c,'Acc_Log_Lasso': AC_Log_Lasso, 'F1_Log_Lasso': F1_Log_Lasso}, ignore_index=True)

In [671]:
#Arr_Log_Lasso

In [672]:
#Busco el "mejor resultado" en el DataFrame creado tras las iteraciones y asigno nuevo nombre para su uso en la comparacion 
#entre los mejores resultados de cada modelo
AC_L_Lasso = Arr_Log_Lasso.nlargest(1,'Acc_Log_Lasso')
ARLasso = AC_L_Lasso.set_index('C')
ARLasso

,Acc_Log_Lasso,F1_Log_Lasso
C,,
500.0,0.921053,0.937778


### Modelo Regresión Logística con Elastic Net

In [673]:
from sklearn.linear_model import ElasticNet

In [674]:
#Buscamos la mejor C de Lasso
#Genero un DataFrame para incluir la totalidad de los resultados de las iteraciones del for
Arr_Log_Elastic = pd.DataFrame(columns=['C','Acc_Log_Elastic', 'F1_Log_Elastic'])

for c in grillaC:
    Log_Elastic = LogisticRegression(l1_ratio=0.5, C=c, max_iter=10000) 
#Entrenamos el modelo de Lasso (Clasificación)
    Log_Elastic.fit(X_valid_train,y_valid_train)            # entrenamos
#Obtenemos los coeficientes del modelo de Lasso (Clasificación)
    y_pred_Elastic = Log_Elastic.predict(X_valid_test)          #pronosticamos
    AC_Log_Elastic = Log_Elastic.score(X_valid_test,y_valid_test)  # evaluamos Accuracy
    F1_Log_Elastic = f1_score(y_valid_test,y_pred_Elastic, average="macro") # evaluamos F1
#Incluyo los resultados de las iteraciones en el DataFrame creado antes del for
    Arr_Log_Elastic = Arr_Log_Elastic.append({'C': c,'Acc_Log_Elastic': AC_Log_Elastic, 'F1_Log_Elastic': F1_Log_Elastic}, ignore_index=True)

In [675]:
#Arr_Log_Elastic

In [676]:
#Busco el "mejor resultado" en el DataFrame creado tras las iteraciones y asigno nuevo nombre para su uso en la comparacion 
#entre los mejores resultados de cada modelo
AC_L_Elastic = Arr_Log_Elastic.nlargest(1,'Acc_Log_Elastic')
ARElastic = AC_L_Elastic.set_index('C')
ARElastic

,Acc_Log_Elastic,F1_Log_Elastic
C,,
25.0,1.0,1.0


### Modelo SVM

In [677]:
# Importamos las librerías para los modelos
from sklearn import svm
from sklearn.svm import LinearSVC

In [678]:
#Buscamos la mejor C de Ridge
#Genero un DataFrame para incluir la totalidad de los resultados de las iteraciones del for
Arr_SVM= pd.DataFrame(columns=['C','Acc_SVM', 'F1_SVM'])

for c in grillaC:
    SVM = LinearSVC(C=c, loss="hinge",dual=True, max_iter=50000,  tol=0.001 )
#Entrenamos el modelo de Ridge (Clasificación)
    SVM.fit(X_valid_train,y_valid_train)            # entrenamos
#Obtenemos los coeficientes del modelo de Lasso (Clasificación)
    y_pred_SVM = SVM.predict(X_valid_test)          #pronosticamos
    AC_SVM = SVM.score(X_valid_test,y_valid_test)  # evaluamos Accuracy
    F1_SVM = f1_score(y_valid_test,y_pred_SVM, average="macro") # evaluamos F1
#Incluyo los resultados de las iteraciones en el DataFrame creado antes del for
    Arr_SVM = Arr_SVM.append({'C': c,'Acc_SVM': AC_SVM, 'F1_SVM': F1_SVM}, ignore_index=True)

In [679]:
#Arr_SVM

In [680]:
#Busco el "mejor resultado" en el DataFrame creado tras las iteraciones y asigno nuevo nombre para su uso en la comparacion 
#entre los mejores resultados de cada modelo
AC_SVM = Arr_SVM.nlargest(1,'Acc_SVM')
SVM = AC_SVM.set_index('C')
SVM

,Acc_SVM,F1_SVM
C,,
0.25,1.0,1.0


### Arbol de desicion

In [681]:
# Importamos las librerías para los modelos
from sklearn.tree import DecisionTreeClassifier  # para Árbol de Decisión
# Importamos las librerías para evaluar
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

In [682]:
#Buscamos la mejor profundidad del Árbol
#Genero un DataFrame para incluir la totalidad de los resultados de las iteraciones del for
Arr_AClass= pd.DataFrame(columns=['Profund','Acc_AClass', 'F1_AClass'])
for n in range(1,Profundidad):
    modelo_AClass = DecisionTreeClassifier(max_depth = n, random_state=123)
#Entrenamos el modelo de Arbol de Decisiciòn (Clasificación)
    modelo_AClass.fit(X_valid_train, y_valid_train)
#Obtenemos los coeficientes del modelo de Arbol de Decisiciòn (Clasificación)
    y_pred_AClass = modelo_AClass.predict(X_valid_test)
    AC_AClass = round(accuracy_score(y_valid_test, y_pred_AClass),3)
    F1_AClass = round(f1_score(y_valid_test,y_pred_AClass, average='macro'),3)
#Incluyo los resultados de las iteraciones en el DataFrame creado antes del for
    Arr_AClass = Arr_AClass.append({'Profund': n,'Acc_AClass': AC_AClass, 'F1_AClass': F1_AClass}, ignore_index=True)

In [683]:
#Arr_AClass

In [684]:
#Busco el "mejor resultado" en el DataFrame creado tras las iteraciones y asigno nuevo nombre para su uso en la comparacion 
#entre los mejores resultados de cada modelo
AClass = Arr_AClass.nlargest(1,'Acc_AClass')
ARClass = AClass.set_index('Profund')
ARClass

,Acc_AClass,F1_AClass
Profund,,
14.0,0.579,0.592


### KNN con weights = 'uniform'

In [685]:
from sklearn import neighbors           # para kNN
import warnings
warnings.simplefilter('ignore')

In [686]:
#Buscamos la mejor cantidad de Vecinos
#Genero un DataFrame para incluir la totalidad de los resultados de las iteraciones del for
Arr_KNClassU = pd.DataFrame(columns=['Vecinos','Acc_KNClass', 'F1_KNClass'])
for k1 in range(1,Vecino):
#Dentro del modelo KNN uso weights='uniform'
    modelo_KNClassU = neighbors.KNeighborsClassifier(n_neighbors = k1,weights='uniform')
#Entrenamos el modelo de KNN
    modelo_KNClassU.fit(X_valid_train,y_valid_train)
#Obtenemos los coeficientes de KNN
    y_pred_KNClassU = modelo_KNClassU.predict(X_valid_test)
    AC_KNClassU = round(accuracy_score(y_valid_test, y_pred_KNClassU),3)
    F1_KNClassU = round(f1_score(y_valid_test,y_pred_KNClassU, average='macro'),3)
#Incluyo los resultados de las iteraciones en el DataFrame creado antes del for
    Arr_KNClassU = Arr_KNClassU.append({'Vecinos': k1,'Acc_KNClass': AC_KNClassU, 'F1_KNClass': F1_KNClassU}, ignore_index=True)

In [687]:
#Arr_KNClassU

In [688]:
#Busco el "mejor resultado" en el DataFrame creado tras las iteraciones y asigno nuevo nombre para su uso en la comparacion 
#entre los mejores resultados de cada modelo
KNClassU = Arr_KNClassU.nlargest(1,'Acc_KNClass')
KNN_unif = KNClassU.set_index('Vecinos')
KNN_unif

,Acc_KNClass,F1_KNClass
Vecinos,,
1.0,1.0,1.0


### KNN con weights = 'distance'

In [689]:
#Buscamos la mejor cantidad de Vecinos
#Genero un DataFrame para incluir la totalidad de los resultados de las iteraciones del for
Arr_KNClass_dist = pd.DataFrame(columns=['Vecinos','Acc_KNClass_dist', 'F1_KNClass_dist'])
for k2 in range(1,Vecino):
#Dentro del modelo KNN uso weights='distance'
    modelo_KNClass_dist = neighbors.KNeighborsClassifier(n_neighbors = k2,weights='distance')
#Entrenamos el modelo de KNN (Clasificación)
    modelo_KNClass_dist.fit(X_valid_train,y_valid_train)
#Obtenemos los coeficientes de KNN (Clasificación)
    y_pred_KNClass_dist = modelo_KNClass_dist.predict(X_valid_test)
    AC_KNClass_dist = round(accuracy_score(y_valid_test, y_pred_KNClass_dist),3)
    F1_KNClass_dist = round(f1_score(y_valid_test,y_pred_KNClass_dist, average='macro'),3)
#Incluyo los resultados de las iteraciones en el DataFrame creado antes del for
    Arr_KNClass_dist = Arr_KNClass_dist.append({'Vecinos': k2,'Acc_KNClass_dist': AC_KNClass_dist, 'F1_KNClass_dist': F1_KNClass_dist}, ignore_index=True)

In [690]:
 #Arr_KNClass_dist

In [691]:
#Busco el "mejor resultado" en el DataFrame creado tras las iteraciones y asigno nuevo nombre para su uso en la comparacion 
#entre los mejores reultados de cada modelo
KNClass_dist = Arr_KNClass_dist.nlargest(1,'Acc_KNClass_dist')
KNN_dist = KNClass_dist.set_index('Vecinos')
KNN_dist

,Acc_KNClass_dist,F1_KNClass_dist
Vecinos,,
1.0,1.0,1.0


## Comparacion entre Modelos

In [692]:
#Uso los DataFrame creados con los "mejores resultados" de cada modelo para su comparacion y determinacion del "modelo definitivo"
Compara = pd.DataFrame({'Modelo': ['Reg Log No Reg','Reg Log Ridge','Reg Log Lasso','Reg Log Elastic','SVM','Arbol de Desic','KNN_unif','KNN_dist'], 'Pramet': [Arr_valid_RLog.iloc[0,0],AC_L_Ridge.iloc[0,0], AC_L_Lasso.iloc[0,0],AC_L_Elastic.iloc[0,0], AC_SVM.iloc[0,0], AClass.iloc[0,0], KNClassU.iloc[0,0], KNClass_dist.iloc[0,0]], 'Acc': [Arr_valid_RLog.iloc[0,1],AC_L_Ridge.iloc[0,1], AC_L_Lasso.iloc[0,1], AC_L_Elastic.iloc[0,1], AC_SVM.iloc[0,1], AClass.iloc[0,1], KNClassU.iloc[0,1], KNClass_dist.iloc[0,1]], 'F1': [Arr_valid_RLog.iloc[0,2],AC_L_Ridge.iloc[0,2], AC_L_Lasso.iloc[0,2], AC_L_Elastic.iloc[0,2], AC_SVM.iloc[0,2], AClass.iloc[0,2], KNClassU.iloc[0,2], KNClass_dist.iloc[0,2]]})
Compara = Compara.set_index('Modelo')
Compara

,Pramet,Acc,F1
Modelo,,,
Reg Log No Reg,xx,0.974000,0.981000
Reg Log Ridge,25.0,1.000000,1.000000
Reg Log Lasso,500.0,0.921053,0.937778
Reg Log Elastic,25.0,1.000000,1.000000
SVM,0.25,1.000000,1.000000
Arbol de Desic,14.0,0.579000,0.592000
KNN_unif,1.0,1.000000,1.000000
KNN_dist,1.0,1.000000,1.000000
